In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf

# Data Preparation

## Stocks

In [31]:
# Use the 'raw' version of the URL
path_stock = 'https://raw.githubusercontent.com/adisorn242/2026_WQU_CapstoneProject/main/set50_stock_monthly_close.csv'

# Import the data
df_stock = pd.read_csv(path_stock, index_col='Date', parse_dates=True)

# Filter from January 2013 onwards
df_stock = df_stock.loc['2012-12-01':]
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 157 entries, 2012-12-01 to 2025-12-01
Data columns (total 50 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADVANC.BK  157 non-null    float64
 1   AOT.BK     157 non-null    float64
 2   AWC.BK     75 non-null     float64
 3   BANPU.BK   157 non-null    float64
 4   BBL.BK     157 non-null    float64
 5   BDMS.BK    157 non-null    float64
 6   BEM.BK     157 non-null    float64
 7   BH.BK      157 non-null    float64
 8   BJC.BK     157 non-null    float64
 9   BTS.BK     157 non-null    float64
 10  CBG.BK     134 non-null    float64
 11  CCET.BK    157 non-null    float64
 12  CENTEL.BK  157 non-null    float64
 13  COM7.BK    125 non-null    float64
 14  CPALL.BK   157 non-null    float64
 15  CPF.BK     157 non-null    float64
 16  CPN.BK     157 non-null    float64
 17  CRC.BK     71 non-null     float64
 18  DELTA.BK   157 non-null    float64
 19  EGCO.BK    157 non-null    floa

In [32]:
# Compute log returns
df_stock_ret = np.log(df_stock / df_stock.shift(1))
df_stock_ret = df_stock_ret.iloc[1:,:]
df_stock_ret.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2013-01-01 to 2025-12-01
Data columns (total 50 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADVANC.BK  156 non-null    float64
 1   AOT.BK     156 non-null    float64
 2   AWC.BK     74 non-null     float64
 3   BANPU.BK   156 non-null    float64
 4   BBL.BK     156 non-null    float64
 5   BDMS.BK    156 non-null    float64
 6   BEM.BK     156 non-null    float64
 7   BH.BK      156 non-null    float64
 8   BJC.BK     156 non-null    float64
 9   BTS.BK     156 non-null    float64
 10  CBG.BK     133 non-null    float64
 11  CCET.BK    156 non-null    float64
 12  CENTEL.BK  156 non-null    float64
 13  COM7.BK    124 non-null    float64
 14  CPALL.BK   156 non-null    float64
 15  CPF.BK     156 non-null    float64
 16  CPN.BK     156 non-null    float64
 17  CRC.BK     70 non-null     float64
 18  DELTA.BK   156 non-null    float64
 19  EGCO.BK    156 non-null    floa

In [33]:
df_stock_ret.index

DatetimeIndex(['2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
               '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
               '2013-09-01', '2013-10-01',
               ...
               '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01',
               '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01',
               '2025-11-01', '2025-12-01'],
              dtype='datetime64[ns]', name='Date', length=156, freq=None)

## T-Bill

In [39]:
# Raw version of the GitHub URL
path_rf = 'https://raw.githubusercontent.com/adisorn242/2026_WQU_CapstoneProject/main/TBILL1M_Daily.csv'

# Import the data and set the Date column as the datetime index
df_tbill_raw = pd.read_csv(path_rf, index_col='Date', parse_dates=True)

# Filter T-Bill data from January 1, 2013 onwards (Daily Data)
df_tbill_raw = df_tbill_raw.loc['2013-01-01':]

In [41]:
# Resample daily data to monthly frequency using the first observation of each month
df_tbill_monthly = df_tbill_raw.resample('MS').first()
df_tbill_monthly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2013-01-01 to 2025-12-01
Freq: MS
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   T-BILL1M  156 non-null    float64
dtypes: float64(1)
memory usage: 2.4 KB


In [46]:
# Check index of Tbill with Stock df
mismatch_count = (df_tbill_monthly.index != df_stock_ret.index).sum()
print(mismatch_count)

0


In [47]:
# Convert annual percentage yield to monthly decimal yield
# 1. Divide by 12 to convert annual rate to monthly rate
# 2. Divide by 100 to convert percentage (e.g., 0.25) to decimal (0.0025)
df_tbill_monthly = (df_tbill_monthly / 12) / 100
df_tbill_monthly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2013-01-01 to 2025-12-01
Freq: MS
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   T-BILL1M  156 non-null    float64
dtypes: float64(1)
memory usage: 2.4 KB


## Gold ETF (KTAM GOLD ETF)

In [61]:
# Download GLD.BK data from Yahoo Finance
df_gld = yf.download('GLD.BK', start='2012-12-01', end='2025-12-31', interval='1mo')

# Keep only the 'Close' column
df_gld = df_gld[['Close']]

# Reset index to move 'Date' out of the index
df_gld = df_gld.reset_index()

# Flatten the MultiIndex columns by keeping only the level where the ticker is located
# and rename the 'Date' column back to a simple string
df_gld.columns = ['Date', 'GLD.BK']

# Set 'Date' back as the index and ensure it is datetime
df_gld = df_gld.set_index('Date')
df_gld.index = pd.to_datetime(df_gld.index)
df_gld.info()

/tmp/ipython-input-4019245837.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_gld = yf.download('GLD.BK', start='2012-12-01', end='2025-12-31', interval='1mo')
[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 157 entries, 2012-12-01 to 2025-12-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GLD.BK  157 non-null    float64
dtypes: float64(1)
memory usage: 2.5 KB


In [62]:
# Calculate log returns for GLD.BK
df_gld_ret = np.log(df_gld / df_gld.shift(1))

# Drop the first row (NaN)
df_gld_ret = df_gld_ret.dropna()
df_gld_ret.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2013-01-01 to 2025-12-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GLD.BK  156 non-null    float64
dtypes: float64(1)
memory usage: 2.4 KB


In [63]:
# Check if the index of df_gld_ret is identical to df_stock_ret
mismatch_check = (df_gld_ret.index != df_stock_ret.index).sum()
mismatch_check

np.int64(0)

# Join data

In [64]:
# Join the three DataFrames into df_ln_ret
# Since all indices are now aligned, we can use pd.concat along axis 1
df_ln_ret = pd.concat([df_stock_ret, df_tbill_monthly, df_gld_ret], axis=1)
df_ln_ret.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2013-01-01 to 2025-12-01
Data columns (total 52 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADVANC.BK  156 non-null    float64
 1   AOT.BK     156 non-null    float64
 2   AWC.BK     74 non-null     float64
 3   BANPU.BK   156 non-null    float64
 4   BBL.BK     156 non-null    float64
 5   BDMS.BK    156 non-null    float64
 6   BEM.BK     156 non-null    float64
 7   BH.BK      156 non-null    float64
 8   BJC.BK     156 non-null    float64
 9   BTS.BK     156 non-null    float64
 10  CBG.BK     133 non-null    float64
 11  CCET.BK    156 non-null    float64
 12  CENTEL.BK  156 non-null    float64
 13  COM7.BK    124 non-null    float64
 14  CPALL.BK   156 non-null    float64
 15  CPF.BK     156 non-null    float64
 16  CPN.BK     156 non-null    float64
 17  CRC.BK     70 non-null     float64
 18  DELTA.BK   156 non-null    float64
 19  EGCO.BK    156 non-null    floa

In [65]:
# Count how many columns have fewer than 156 non-null observations
columns_less_than_156 = (df_ln_ret.count() < 156).sum()

print(f"Number of columns with less than 156 rows: {columns_less_than_156}")

Number of columns with less than 156 rows: 13


# Export data

In [67]:
if False:
    # Export the joined log returns to a CSV file
    df_ln_ret.to_csv('df_ln_ret.csv', index=True)